<a href="https://colab.research.google.com/github/Eugene2705/MITx_Supply_Chain_Optimisation/blob/main/SCM_275x_Decisions_on_Demand_to_Serve_Python_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCM.275x - Advanced Supply Chain Systems Planning and Network Design
# **Decisions on Demand to Serve - Python Exercise**

### *Before starting, make sure to save a copy of this notebook to your Google Drive!*

## **Initialization**

In [1]:
# Import all required packages

import pandas as pd                   # For data manipulation and analysis
import gurobipy as grb                # Gurobi optimization library for solving mathematical models
import folium                         # For creating interactive maps
import folium.plugins as plugins      # Additional plugins for folium
from geopy.distance import geodesic   # For calculating geodesic distances between two points


## **Helper functions**

### **Ploting nodes on a map**

In [2]:
# Defining a function to plot nodes on a map using folium

def plot_nodes(map,                         # Folium map object to plot the nodes on
               nodes,                       # Dictionary of node objects where each node contains attributes like latitude and longitude
               icon,                        # Icon symbol to use for the markers on the map
               active_color,                # Color of the marker icon for active nodes
               background_color,            # Background color of the marker icon
               inactive_color = 'grey',     # Color of the marker icon for inactive nodes
               ):

    # Loop through each node in the dictionary
    for node in nodes.values():

        # Create a folium marker
        marker = folium.Marker(
            location=[node.lat, node.lon],              # Set the marker's location
            popup = (node.ID + "-" + node.name),        # Create a marker popup with the node ID and name
            icon=plugins.BeautifyIcon(                  # Create a marker's icon
                icon=icon,
                icon_shape="circle",
                text_color=active_color if node.active == True else inactive_color,
                border_color=active_color if node.active == True else inactive_color,
                background_color=background_color,
            )
        )

        # Add a folium marker to the map
        marker.add_to(map)


### **Computing geodesic distance**

In [3]:
# Defining a function for computing geodesic distances between two locations

def compute_geodesic_distance(origin,       # Origin node object
                              destination,  # Destination node
                              unit='km'):   # Unit ('km' or 'mi')

    # Extract coordinates (latitude and longitude) from origin and destination
    origin_coordinates = [origin.lat, origin.lon]
    destination_coordinates = [destination.lat, destination.lon]

    # Compute distance based on the specified unit
    if unit == 'km':
        distance = geodesic(origin_coordinates, destination_coordinates).km  # Compute distance in kilometers
    elif unit == 'mi':
        distance = geodesic(origin_coordinates, destination_coordinates).mi  # Compute distance in miles

    return distance  # Return the calculated distance


### **Ploting flows of different products on a map**

In [4]:
# Defining a function to plot flows on a map using folium

def plot_flows_products(map,          # Folium map object where flows will be plotted.
               vars,                  # Dictionary of decision variables from the optimization model
               nodes,                 # Dictionary of node objects
               product_colors,        # Dictionary representing colors of the lines representing flows of different products
               max_width = 30,        # Maximum line width for the flows, default is 30
               opacity = 0.5):        # Opacity of the lines, default is 0.5

    # Find the maximum flow value to normalize line widths
    max_val = max([var.X for (node1_key, node2_key, product), var in vars.items()])

    # Iterate over flow decision variables (keys represent node pairs)
    for (node1_key, node2_key, product), var in vars.items():

        # Plot only positive flows
        if var.X > 0:

          # Get the coordinates of the nodes for plotting the line
          points = [[nodes[node1_key].lat, nodes[node1_key].lon],
                      [nodes[node2_key].lat, nodes[node2_key].lon]]


          # Add a PolyLine to the map to represent the flow between the nodes
          folium.PolyLine(points,
                            color=product_colors[product],              # Set the color of the line
                            weight=var.X / max_val * max_width,         # Normalize line width based on flow value
                            opacity=opacity,                            # Set line opacity
                            popup=var.X).add_to(map)                    # Show the flow value in a popup on the map

## **Data setup and preprocessing**

### **Nodes**

#### Reading input files

In [5]:
# File containing customer data
customer_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/customers.csv'

# Loading customer data into a pandas DataFrame
customers_df = pd.read_csv(customer_data_file)

# Displaying the first few rows of the DataFrame to verify the data
customers_df.head()

,ID,name,lat,lon
0,c1,Charlotte,35.2083,-80.8303
1,c2,Milwaukee,43.0642,-87.9675
2,c3,Providence,41.8230,-71.4187
3,c4,Jacksonville,30.3322,-81.6749
4,c5,Nashville,36.1715,-86.7842


In [6]:
# File containing demand data
demand_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/demand_data.csv'

# Loading demand data file into a pandas DataFrame
demand_data_df = pd.read_csv(demand_data_file)

# Displaying the first few rows of the DataFrame to verify the data
demand_data_df.head()

,customerID,product,demand,price
0,c1,product_a,14300,50
1,c2,product_a,12900,50
2,c3,product_a,12900,50
3,c4,product_a,12900,50
4,c5,product_a,11800,50


In [7]:
# File containing warehouse data
warehouse_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/warehouses.csv'

# Loading warehouse data into a pandas DataFrame
warehouses_df = pd.read_csv(warehouse_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehouses_df.head()

,ID,name,lat,lon,capacity,fixed_cost
0,w1,Detroit,42.3834,-83.1024,100000,1000000
1,w2,Denver,39.7620,-104.8758,100000,1000000
2,w3,Portland,45.5371,-122.6500,100000,1000000
3,w4,Minneapolis,44.9635,-93.2678,100000,1000000
4,w5,Orlando,28.4773,-81.3370,100000,1000000


In [8]:
# File containing landed cost data
warehousing_data_file = 'https://raw.githubusercontent.com/scm275/problem_sets_scm275/main/demand_decisions/warehousing_data.csv'

# Loading landed cost data into a pandas DataFrame
warehousing_df = pd.read_csv(warehousing_data_file)

# Displaying the first few rows of the DataFrame to verify the data
warehousing_df.head()

,warehouseID,productID,landed_cost
0,w1,product_a,40
1,w1,product_b,56
2,w2,product_a,41
3,w2,product_b,58
4,w3,product_a,39


#### Definition of Classes

In [9]:
# Class representing a Customer object

class Customer():
    def __init__(self, ID, name, lat, lon):
        self.ID = ID                    # Customer's ID
        self.name = name                # Customer's name
        self.lat = lat                  # Customer's latitude
        self.lon = lon                  # Customer's longitude

        self.demand = dict()            # Initializing an empty dictionnary to store customers's demand per product
        self.selling_price = dict()     # Initializing an empty dictionnary to store customers's demand per product

        self.active = True              # Initializing node as active


In [10]:
# Class representing a Warehouse object

class Warehouse():
    def __init__(self, ID, name, lat, lon, fixed_cost, capacity):
        self.ID = ID                  # Warehouse's ID
        self.name = name              # Warehouse's name
        self.lat = lat                # Warehouse's latitude
        self.lon = lon                # Warehouse's longitude
        self.fixed_cost = fixed_cost  # Warehouse's fixed cost
        self.capacity = capacity      # Warehouse's capacity

        self.landed_cost = dict()     # Initializing an empty dictionnary to store warehouse's landed cost per product

        self.active = True            # Initializing node as active


#### Creating node objects

In [11]:
# Initializing an empty dictionary to store node objects
nodes = dict()

In [12]:
# Creating a dictionary of customer objects
customers = dict()
for i, row in customers_df.iterrows():
    customers[row['ID']] = Customer(ID=row['ID'],           # Customer's ID
                                    name=row['name'],       # Customer's name
                                    lat=row['lat'],         # Customer's latitude
                                    lon=row['lon'])         # Customer's longitude

# Merging the customers dictionary into the existing nodes dictionary
nodes = {**nodes, **customers}

In [13]:
# Associating demand data to customers
for i, row in demand_data_df.iterrows():
    customers[row['customerID']].demand[row['product']] = row['demand']
    customers[row['customerID']].selling_price[row['product']] = row['price']

In [14]:
# Visualizing an example of customer demand
customers['c1'].demand

{'product_a': 14300, 'product_b': 21450}

In [15]:
# Visualizing an example of customer selling price
customers['c1'].selling_price

{'product_a': 50, 'product_b': 90}

In [16]:
# Creating a dictionary of warehouse objects
warehouses = dict()
for i, row in warehouses_df.iterrows():
    warehouses[row['ID']] = Warehouse(ID = row['ID'],               # Warehouse's ID
                                    name = row['name'],             # Warehouse's name
                                    lat = row['lat'],               # Warehouse's latitude
                                    lon = row['lon'],               # Warehouse's longitude
                                    fixed_cost = row['fixed_cost'], # Warehouse's fixed cost
                                    capacity = row['capacity'])     # Warehouse's fixed cost

# Merging the warehouse dictionary into the existing nodes dictionary
nodes = {**nodes, **warehouses}

In [17]:
# Associating warehousing data to warehouses
for i, row in warehousing_df.iterrows():
    warehouses[row['warehouseID']].landed_cost[row['productID']] = row['landed_cost']

#### Visualizing node objects

In [18]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map


### **Arcs**

#### Arc distances

In [19]:
# Creating a dictionary containing distances between warehouses and customers

distances = dict()
for w, warehouse in warehouses.items():
  for c, customer in customers.items():
      distances[w, c] = compute_geodesic_distance(origin = warehouse, destination = customer, unit = 'km')

#### Arc costs

In [20]:
cost_unit_km_warehouse_customer = 0.01    # Cost per unit per kilometer from warehouse to customer

# Creating a dictionary containing unit costs between suppliers and warehouses, and between warehouses and customers
unit_cost = dict()

for w, warehouse in warehouses.items():                                                        # Iterate over warehouses
    for c, customer in customers.items():                                                      # Iterate over customers
        unit_cost[w, c] = distances[w, c] * cost_unit_km_warehouse_customer                    # Calculate unit cost as distance multiplied by cost per km (warehouse to customer)


### **Products**

In [21]:
# Defining a list of products to consider in the model
products = ['product_a', 'product_b']

## **Optimization model - Profit Maximization**

### **Creating and solving the optimization model**

***❗Task 1: Run the code to obtain the value of profit margins for the different warehouses. Note the value obtained for the warehouse in Orlando.***

***❗Task 2: Modify the optimization model to introduce an additional constraint ensuring that the profit margin for each individual warehouse is greater than or equal to 30%***

In [54]:
# Initializing the optimization model for profit maximization
model = grb.Model("Profit maximization model")

# Creating binary decision variables to determine if a warehouse is open
wh_location_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over warehouses
    wh_location_vars[w] = model.addVar(vtype=grb.GRB.BINARY,
                               name = "wh_location_{0}".format(w))

# Creating continuous decision variables for flow from warehouses to customers
flow_vars = dict()
for w, warehouse in warehouses.items():  # Iterate over warehouses
    for c, customer in customers.items():  # Iterate over customers
        for p in products:  # Iterate over products
            flow_vars[w, c, p] = model.addVar(vtype=grb.GRB.CONTINUOUS,
                               name = "flow_{0}_{1}_{2}".format(w, c, p))

# Creating the objective function

# Calculating revenue from selling products to customers
revenue = grb.quicksum(flow_vars[w, c, p] * cust.selling_price[p]
                        for w, warehouse in warehouses.items()
                        for c, cust in customers.items()
                        for p in products)

# Calculating fixed costs for opening warehouses
fixed_cost_warehouses = grb.quicksum(wh_location_vars[w]
                          * warehouse.fixed_cost
                          for w, warehouse in warehouses.items())

# Calculating transportation costs for flows from warehouses to customers
cost_warehouse_customer = grb.quicksum(flow_vars[w, c, p] * unit_cost[w, c]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Calculating landed costs for products in each warehouse
landed_cost = grb.quicksum(flow_vars[w, c, p] * warehouse.landed_cost[p]
                        for w, warehouse in warehouses.items()
                        for c, customer in customers.items()
                        for p in products)

# Total profit is revenue minus fixed costs, transportation costs, and landed costs
total_profit = revenue - fixed_cost_warehouses - cost_warehouse_customer - landed_cost

# Setting the objective to maximize total profit
model.setObjective(total_profit, grb.GRB.MAXIMIZE)

# Adding demand constraints (flow to each customer must not exceed their demand)
for c, customer in customers.items():
    for p in products:
        model.addConstr(grb.quicksum(flow_vars[w, c, p] for w, warehouse in warehouses.items()) <= customer.demand[p])

# Adding warehouse capacity constraints (flow out of each warehouse must not exceed its capacity)
for w, warehouse in warehouses.items():
    model.addConstr(grb.quicksum(flow_vars[w, c, p] for c, customer in customers.items() for p in products) <= wh_location_vars[w] * warehouse.capacity)




# ADDITION 1: In this code, we consider individual warehouses and compute different costs and revenue expressions
# For example, "warehouse.fixed_cost_expression" is the expression of the fixed cost incurred at the level of each warehouse
# If the warehouse is active (i.e., wh_location_vars[w] = 1, it is equal to the 'warehouse.fixed_cost' parameter)
# Otherwise, it is equal to zero. Additionally, we compute the values of transportation cost, landed cost and revenue.


for w, warehouse in warehouses.items():
  # Fixed cost incurred at each warehouse (i.e. only account for the value if the warehouse is active)
  warehouse.fixed_cost_expression = wh_location_vars[w] * warehouse.fixed_cost

  # Transportation cost incurred at each warehouse
  warehouse.transportation_cost_expression = grb.quicksum(flow_vars[w, c, p] * unit_cost[w, c]
                        for c, customer in customers.items()
                        for p in products)

  # Landed cost incurred at each warehouse
  warehouse.landed_cost_expression = grb.quicksum(flow_vars[w, c, p] * warehouse.landed_cost[p]
                        for c, customer in customers.items()
                        for p in products)

  # Total cost incurred at each warehouse
  warehouse.total_cost_expression = warehouse.fixed_cost_expression + warehouse.landed_cost_expression + warehouse.transportation_cost_expression

  # Revenue from each warehouse
  warehouse.revenue_expression = grb.quicksum(flow_vars[w, c, p] * cust.selling_price[p]
                        for c, cust in customers.items()
                        for p in products)




### Task 2: Introduce an additional constraint ensuring that the profit margin for each individual warehouse is greater than or equal to 30

for w, warehouse in warehouses.items():
 model.addConstr(warehouse.revenue_expression- warehouse.total_cost_expression >= warehouse.revenue_expression * 0.3)

# Solving the optimization model
model.optimize()





# ADDITION 2: In this code, we comppute the profit margin of each warehouse using the following formula
# profit margin = (total revenue - total cost) / (total revenue)
# We then print the profit margin for each warehouse

print("----")
print('Printing the profit margins of warehouses:')
for w, warehouse in warehouses.items():
  if warehouse.revenue_expression.getValue() > 0:
    warehouse.profit_margin = (warehouse.revenue_expression.getValue() - warehouse.total_cost_expression.getValue()) / warehouse.revenue_expression.getValue()
    print(warehouse.name, round(warehouse.profit_margin, 3))



Gurobi Optimizer version 13.0.0 build v13.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-1345U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 80 rows, 355 columns and 1060 nonzeros (Max)


Model fingerprint: 0x558a7386
Model has 355 linear objective coefficients
Variable types: 350 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+06]
  Objective range  [2e-02, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 2e+04]
Found heuristic solution: objective -0.0000000
Presolve removed 35 rows and 256 columns
Presolve time: 0.00s
Presolved: 45 rows, 99 columns, 291 nonzeros
Variable types: 97 continuous, 2 integer (2 binary)

Root relaxation: objective 5.038554e+06, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5038554.29    0    1   -0.00000 5038554.29      -     -    0s
H    0     0                    3617118.2536 5038554.29  39.3%     -    0s
     0     0 3617118.25    0    1 3617118.25 3617118.25  0.00%     -    0s

Cutting planes:
  Implied bound: 7
  Flow

### **Solution visualization and analysis**

In [41]:
sum([w.revenue_expression.getValue() for w in warehouses.values()]) - sum([w.total_cost_expression.getValue() for w in warehouses.values()])

8185244.825812757

In [52]:
# Create a new map centered on Europe with a zoom level of 5
map = folium.Map([40, -100.0], zoom_start=5)

# Plot customer locations with a store icon, green color, and yellow background
plot_nodes(map=map, nodes=customers, icon='store', active_color='green', background_color='yellow')

# Plot warehouse locations with a warehouse icon, blue color, and yellow background
plot_nodes(map=map, nodes=warehouses, icon='warehouse', active_color='blue', background_color='yellow')

# Define different colors for plotting products
colors = {'product_a': 'red',
          'product_b': 'green'}

# Plot flows on the map
plot_flows_products(map = map, vars = flow_vars, nodes = nodes, product_colors = colors, opacity = 0.3)

# Add a tile layer for better map visualization (cartodbpositron theme)
folium.TileLayer('cartodbpositron').add_to(map)

# Display the map with all the plotted data
map
